In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "gdsc1"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]

load gdsc1


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)

    val_labels = sampler.test_data[sampler.test_mask]

    if len(np.unique(val_labels)) < 2:
        print(f"Target {target_index} skipped: Validation set has only one class.")
        return None, None

    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [7]:
from joblib import Parallel, delayed
from tqdm import tqdm

n_kfold = 1
n_jobs = 50  # 並列数


def process_iteration(dim, target_index, seed, args):
    """各反復処理をカプセル化した関数"""
    if dim:
        if drug_sum[target_index] < 10:
            return None, None
    else:
        if cell_sum[target_index] < 10:
            return None, None

    fold_results = []
    for fold in range(n_kfold):
        true_data, predict_data = nihgcn_new(
            cell_exprs=exprs,
            drug_finger=drug_finger,
            res_mat=res,
            null_mask=null_mask,
            target_dim=dim,
            target_index=target_index,
            evaluate_fun=roc_auc,
            args=args,
            seed=seed,
        )
        fold_results.append((true_data, predict_data))

    return fold_results


# 並列処理の実行
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()

for dim in target_dim:
    # 全タスクを事前に生成
    tasks = [
        (dim, target_index, seed, args)
        for seed, target_index in enumerate(np.arange(res.shape[dim]))
    ]

    # 並列実行（プログレスバー付き）
    results = Parallel(n_jobs=n_jobs, verbose=0, prefer="threads")(
        delayed(process_iteration)(*task)
        for task in tqdm(tasks, desc=f"Processing dim {dim}")
    )

    # 結果の結合
    for fold_results in results:
        if fold_results is None:
            continue
        for true_data, predict_data in fold_results:
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)],
                ignore_index=True,
                copy=False,  # メモリ節約のため
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)],
                ignore_index=True,
                copy=False,
            )


Processing dim 0:   0%|          | 0/916 [00:00<?, ?it/s]

epoch:   0 loss:0.699423 auc:0.4926
epoch:   0 loss:0.702522 auc:0.6013
epoch:   0 loss:0.700659 auc:0.3660
epoch:   0 loss:0.709158 auc:0.5193
epoch:   0 loss:0.700974 auc:0.5177
epoch:   0 loss:0.704072 auc:0.4011
epoch:   0 loss:0.697085 auc:0.5360
epoch:   0 loss:0.704606 auc:0.4568
epoch:   0 loss:0.700472 auc:0.4269
epoch:   0 loss:0.707393 auc:0.4697
epoch:   0 loss:0.699546 auc:0.6096
epoch:   0 loss:0.694028 auc:0.4909
epoch:   0 loss:0.713105 auc:0.2761
epoch:   0 loss:0.697562 auc:0.5113
epoch:   0 loss:0.698030 auc:0.5469
epoch:   0 loss:0.705123 auc:0.4761
epoch:   0 loss:0.701771 auc:0.4748
epoch:   0 loss:0.701083 auc:0.5163
epoch:   0 loss:0.693330 auc:0.4740
epoch:   0 loss:0.702457 auc:0.4695
epoch:   0 loss:0.708718 auc:0.5615
epoch:   0 loss:0.694293 auc:0.4471
epoch:   0 loss:0.693570 auc:0.4631
epoch:   0 loss:0.689798 auc:0.5079
epoch:   0 loss:0.702347 auc:0.5093
epoch:   0 loss:0.694964 auc:0.5656
epoch:   0 loss:0.690334 auc:0.5189
epoch:   0 loss:0.698758 auc

KeyboardInterrupt: 

epoch: 140 loss:0.132730 auc:0.9804
epoch:  40 loss:0.179879 auc:0.9563
epoch:  40 loss:0.178180 auc:0.9796
epoch:  40 loss:0.179890 auc:0.9831
epoch:  40 loss:0.177763 auc:0.9585
epoch:  40 loss:0.179032 auc:0.9655
epoch:  40 loss:0.180500 auc:0.9550
epoch:  80 loss:0.156798 auc:0.9474
epoch:  40 loss:0.178979 auc:0.9633
epoch: 160 loss:0.124295 auc:0.9519
epoch: 160 loss:0.125164 auc:0.9773
epoch:  40 loss:0.179129 auc:0.9735
epoch:  40 loss:0.179352 auc:0.9478
epoch:  40 loss:0.177163 auc:0.9640
epoch:  40 loss:0.177447 auc:0.9863
epoch:  60 loss:0.165400 auc:0.9823
epoch:  60 loss:0.166888 auc:0.9543
epoch:  60 loss:0.165552 auc:0.9681
epoch: 100 loss:0.147454 auc:0.9489
epoch:  60 loss:0.166269 auc:0.9838
epoch:  60 loss:0.164691 auc:0.9568


In [ ]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")